In [148]:
import pandas as pd
import numpy as np
import itertools
from collections import Counter
from IPython.display import display, HTML

HTML('<style>.output {flex-direction: row;}</style>')

### toy example: count

In [342]:
# intersection of all expression vertices between cohort-level CLOvE pairs and the pairs in a sample (with that specific context)
cohort = pd.DataFrame(np.random.randint(0,10,size=(16, 1)), columns=['score'])
cohort['exp'] = ['a','b','c','d','a','b','c','d','a','b','c','d','a','b','c','d']
cohort['cnv'] = ['A','A','A','A','B','B','B','B','C','C','C','C','D','D','D','D']

cnv_s = pd.DataFrame(np.random.randint(0,2,size=(4, 4)), columns=['p1','p2','p3','p4'])
exp_s = pd.DataFrame(np.random.rand(4, 4), columns=['p1','p2','p3','p4'])

In [343]:
exp_s

,p1,p2,p3,p4
0,0.903135,0.221591,0.727339,0.617464
1,0.207924,0.028553,0.991886,0.289566
2,0.874191,0.064179,0.047417,0.648652
3,0.336999,0.585830,0.146329,0.127086


In [352]:
exp_s = exp_s[exp_s > 0.4]

In [353]:
cnv_s['genes'] = ['A','B','C','D']
cnv_s.set_index(keys='genes', drop=True,  inplace=True)

exp_s['genes'] = ['a','b','c','d']
exp_s.set_index(keys='genes', drop=True,  inplace=True)

hits_s = cohort[cohort['score'] > 4]  # models significance cutoff
hits_s['pair'] = hits_s['exp'] + hits_s['cnv']

C:\Users\test\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [354]:
display(hits_s)
display(cnv_s)
display(exp_s)

,score,exp,cnv,pair
0,9,a,A,aA
1,8,b,A,bA
3,5,d,A,dA
4,9,a,B,aB
5,5,b,B,bB
6,7,c,B,cB
9,6,b,C,bC
10,6,c,C,cC
11,8,d,C,dC
13,5,b,D,bD


,p1,p2,p3,p4
genes,,,,
A,1,0,0,1
B,1,1,0,1
C,0,1,0,0
D,0,0,1,0


,p1,p2,p3,p4
genes,,,,
a,0.903135,NaN,0.727339,0.617464
b,NaN,NaN,0.991886,NaN
c,0.874191,NaN,NaN,0.648652
d,NaN,0.58583,NaN,NaN


In [350]:
vulnerability_vector_count(exp_s, cnv_s, hits_s)

p1
['aA', 'aB', 'cA', 'cB', 'dA', 'dB']
['d', 'a', 'a', 'c'] 

p2
['dB', 'dC']
['d'] 

p3
['aD', 'bD']
['b'] 

p4
['aA', 'aB', 'cA', 'cB']
['a', 'a', 'c'] 



,p1,p2,p3,p4
genes,,,,
a,2,0,0,2
b,0,0,1,0
c,1,0,0,1
d,1,1,0,0


In [360]:
vulnerability_vector_count(exp_s, cnv_s, hits_s)

,p1,p2,p3,p4
genes,,,,
a,2,0,0,2
b,0,0,1,0
c,1,0,0,1
d,0,1,0,0


In [363]:
def vulnerability_vector_count(exp, cnv, hits):
    """Counts meaningful clDEGs (copy-loss differential expression genes) from loss contexts
    
    :param exp: df, gene x sample microarray or RNAseq, nan where no expression counts exist
    :param cnv: df, gene x sample calls from GISTIC, 0=WT 1=loss
    :param hits: df, pair x calculations from clove output, cutoff to sig, with 'pair' col
    
    returns gene x sample df of counts of significant expression linkages"""
    
    pat_essential = pd.DataFrame(0, index=exp.index, columns=exp.columns)
    
    for col in exp.columns:
        
        g_exp = exp[col].dropna().index.tolist()  # non-NaN genes from exp matrix
        g_cnv = cnv[cnv[col] == 1][col].index.tolist()  # genes with value 1 from cnv matrix
        
        df = hits.loc[(hits['exp'].isin(g_exp)) & (hits['cnv'].isin(g_cnv))]
        df = pd.DataFrame(df['exp'].value_counts())
        
        if df.shape[0] != 0:
            df.columns=[col]
            pat_essential[col] = df
            
    pat_essential.replace(np.nan, 0, inplace=True)
    return pat_essential.astype(np.int32)

In [317]:
def vulnerability_vector_count_himem(exp, cnv, hits):
    """Counts meaningful gene expression linkages from loss contexts
    
    :param exp: df, gene x sample microarray or RNAseq, nan where no expression counts exist
    :param cnv: df, gene x sample calls from GISTIC, 0=WT 1=loss
    :param hits: df, pair x calculations from clove output, cutoff to sig, with 'pair' col
    
    returns gene x sample df of counts of significant expression linkages"""
    
    pat_essential = pd.DataFrame(0, index=exp.index, columns=exp.columns)
    for col in exp.columns:
        g_exp = exp[col].dropna().index.tolist()  # non-NaN genes from exp matrix
        g_cnv = cnv[cnv[col] == 1][col].index.tolist()  # genes with value 1 from cnv matrix
        
        # building all combinations of pairs from exp and cnv is not needed (Mem Err)
        pairs_list = [''.join(pair) for pair in itertools.product(g_exp, g_cnv)]  # form column as joined pair
        essential = [g[0] for g in set(hits['pair']).intersection(pairs_list)]
        print(col)
        print(pairs_list)
        print(essential,'\n')
        
        # try instead to pull from hits['pair']
        df = pd.DataFrame.from_dict(dict(Counter(essential)), orient='index')
        if df.shape[0] != 0:
            df.columns=[col]
            pat_essential[col] = df
    pat_essential.replace(np.nan, 0, inplace=True)
    return pat_essential.astype(np.int32)

In [281]:
def prepare_vv(exp, cnv, cloves, sig=0.01):
    """Prepare CLoVE computations df for vulnerability vector
    
    :param cloves: df, pair x calculations from clove output
    :param exp: df, gene x sample microarray or RNAseq, nan where no expression counts exist
    :param cnv: df, gene x sample calls from GISTIC, 0=WT 1=loss
    :param sig: float, p-value threshold for signifiance, default=0.01
    
    returns cutoff'd pair x calculations df from clove output with 'pair' col
    """
    
    if 'pair' not in cloves.columns:
        cloves['pair'] = cloves['exp'] + cloves['cnv']
#         for df in [exp, cnv]:
#             if df.index.dtype != 'object':
#                 df.set_index(keys='gene_id', drop=True, inplace=True)
    if exp.index.dtype != 'object':
        exp.set_index(keys='gene_id', drop=True, inplace=True)
    if cnv.index.dtype != 'object':
        cnv.set_index(keys='gene_id', drop=True, inplace=True)
    
    return exp, cnv, cloves[cloves['np_p_w'] <= sig]


In [361]:
exp = pd.read_csv('brca_exp.tab.gz', sep='\t', compression='gzip')
cnv = pd.read_csv('brca_cnv_het.tab.gz', sep='\t', compression='gzip')
scores = pd.read_csv('brca_1M_clove.tab.gz', sep='\t', compression='gzip')

In [364]:
exp, cnv, scores = prepare_vv(exp, cnv, scores, sig=0.001)
counts = vulnerability_vector_count(exp, cnv, scores)

In [375]:
counts.to_csv('brca_het_clDEG_count.tab.gz', sep='\t', compression='gzip')

In [367]:
counts.head()

,TCGA-AR-A24V-01A,TCGA-A7-A26H-01A,TCGA-EW-A6SB-01A,TCGA-C8-A137-01A,TCGA-S3-A6ZG-01A,TCGA-BH-A42T-01A,TCGA-OL-A5RU-01A,TCGA-AQ-A54N-01A,TCGA-AO-A0J3-01A,TCGA-A7-A5ZV-01A,...,TCGA-BH-A18L-01A,TCGA-AO-A1KT-01A,TCGA-AR-A24S-01A,TCGA-AO-A1KS-01A,TCGA-GM-A3XN-01A,TCGA-A2-A25E-01A,TCGA-E2-A15O-01A,TCGA-AR-A5QP-01A,TCGA-E9-A24A-01A,TCGA-AN-A0FJ-01A
gene_id,,,,,,,,,,,,,,,,,,,,,
A1BG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A1CF,0,0,1,1,0,1,0,2,2,0,...,0,0,2,0,0,1,0,0,0,2
A2BP1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A2LD1,2,2,4,3,0,1,0,4,3,0,...,0,1,3,1,0,1,0,0,0,3
A2M,0,0,2,0,1,1,0,2,2,0,...,1,0,2,1,1,0,1,1,1,2


In [371]:
pd.DataFrame(counts.sum(axis=1).sort_values(ascending=False)).head(20)

,0
gene_id,
CDK1,14148
TROAP,13866
STIP1,13756
H2AFZ,13664
GINS1,13319
CCNB1,13044
CDC45,13016
CENPI,12877
KIAA0247,12772


In [373]:
counts.shape


(20502, 1066)

In [374]:
scores.shape

(160654, 16)